# Распознавание собак и кошек на изображениях с помощью признаков, извлеченных VGG16

**Источник данных** - соревнования Kaggle [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/data).

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования.


In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam
import numpy as np

In [0]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 10
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

## Загружаем предварительно обученную нейронную сеть

In [3]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


## Готовим генераторы данных для извлечения признаков

In [0]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [5]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 17500 images belonging to 2 classes.


In [6]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 3750 images belonging to 2 classes.


In [7]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

Found 3750 images belonging to 2 classes.


## Генерируем признаки и сохраняем их в файлы

Признаки для изображений из набора данных для обучения

In [0]:
features_train = vgg16_net.predict_generator(
        train_generator, nb_train_samples // batch_size)

In [0]:
np.save(open('features_train.npy', 'wb'), features_train)

Признаки для изображений из проверочного набора данных

In [0]:
features_val = vgg16_net.predict_generator(
        val_generator, nb_validation_samples // batch_size)

In [0]:
np.save(open('features_val.npy', 'wb'), features_val)

Признаки для изображений из тестового набора данных

In [0]:
features_test = vgg16_net.predict_generator(
        test_generator, nb_test_samples // batch_size)

In [0]:
np.save(open('features_test.npy', 'wb'), features_test)

Генерируем метки для трех наборов данных

In [0]:
labels_train =  np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

In [0]:
labels_val =  np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

In [0]:
labels_test =  np.array(
        [0] * (nb_test_samples // 2) + [1] * (nb_test_samples // 2))

## Загружаем признаки из файлов

In [0]:
features_train = np.load(open('features_train.npy', 'rb'))
features_val = np.load(open('features_val.npy', 'rb'))
features_test = np.load(open('features_test.npy', 'rb'))

## Создаем полносвязную сверточную нейронную сеть для классификации извлеченных из изображения признаков

In [18]:
model = Sequential()
model.add(Flatten(input_shape=features_train.shape[1:]))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(optimizer='Adam',
              loss='binary_crossentropy', metrics=['accuracy'])

Обучаем модель на признаках, которые извлекли из изображений

In [20]:
model.fit(features_train, labels_train,
              epochs=15,
              batch_size=64,
              validation_data=(features_val, labels_val), verbose=2)

Train on 17500 samples, validate on 3750 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
 - 3s - loss: 0.3360 - acc: 0.8611 - val_loss: 0.2924 - val_acc: 0.8701
Epoch 2/15
 - 2s - loss: 0.2283 - acc: 0.8999 - val_loss: 0.2341 - val_acc: 0.8987
Epoch 3/15
 - 2s - loss: 0.2036 - acc: 0.9126 - val_loss: 0.2200 - val_acc: 0.9096
Epoch 4/15
 - 2s - loss: 0.1850 - acc: 0.9210 - val_loss: 0.2368 - val_acc: 0.9035
Epoch 5/15
 - 2s - loss: 0.1718 - acc: 0.9291 - val_loss: 0.2081 - val_acc: 0.9131
Epoch 6/15
 - 2s - loss: 0.1600 - acc: 0.9331 - val_loss: 0.2061 - val_acc: 0.9163
Epoch 7/15
 - 2s - loss: 0.1506 - acc: 0.9378 - val_loss: 0.2504 - val_acc: 0.9016
Epoch 8/15
 - 2s - loss: 0.1325 - acc: 0.9457 - val_loss: 0.2196 - val_acc: 0.9163
Epoch 9/15
 - 2s - loss: 0.1240 - acc: 0.9498 - val_loss: 0.2293 - val_acc: 0.9179
Epoch 10/15
 - 2s - loss: 0.1131 - acc: 0.9554 - val_loss: 0.2326 - val_acc: 0.9152
Epoch 11/15
 - 2s - loss: 0.1067 - acc: 0.9573 - val_loss: 0.2397 - val_

In [21]:
scores = model.evaluate(features_test, labels_test, verbose=1)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

3750/3750 [==============================] - 0s 63us/sample - loss: 0.2955 - acc: 0.9056
Аккуратность на тестовых данных: 90.56%
